In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

missing_keys = [
    key for key in ["OPENAI_API_KEY", "GOOGLE_API_KEY", "ANTHROPIC_API_KEY", "DEEPSEEK_API_KEY"]
    if not os.getenv(key)
]
if missing_keys:
    raise ValueError(f"Missing keys in .env: {', '.join(missing_keys)}")

openai = OpenAI(api_key=openai_api_key)
google.generativeai.configure(api_key=google_api_key)
claude = anthropic.Anthropic(api_key=anthropic_api_key)

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

system_message = "You are a witty and creative assistant known for hilarious one-liners."
user_prompt = "What's your absolute favorite one-liner?"

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
]

print("----- From ChatGPT -----")
try:
    openai_response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=prompts,
        temperature=0.7,
        stream=True
    )
    for chunk in openai_response:
        content = chunk.choices[0].delta.content
        if content:
            print(content, end='', flush=True)
except Exception as e:
    print(f"\nError from OpenAI: {e}")

# gemini api is different
print("\n\n----- From Gemini -----")
try:
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-2.0-flash',
        system_instruction=system_message
    )
    gemini_response = gemini.generate_content(user_prompt, stream=True)
    for chunk in gemini_response:
        print(chunk.text, end='', flush=True)
except Exception as e:
    print(f"\nError from Gemini: {e}")

# you can use gemini with the openai library
print("\n\n----- From Gemini through OpenAI -----")
try:
    gemini_openai_response = gemini_via_openai_client.chat.completions.create(
        model="gemini-2.0-flash",
        messages=prompts,
        temperature=0.7,
        stream=True
    )
    for chunk in gemini_openai_response:
        content = chunk.choices[0].delta.content
        if content:
            print(content, end='', flush=True)
except Exception as e:
    print(f"\nError from Gemini via OpenAI: {e}")

print("\n\n----- From Claude -----")
try:
    claude_response = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=200,
        temperature=0.7,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt}],
    )
    with claude_response as stream:
        for text in stream.text_stream:
            print(text, end="", flush=True)
except Exception as e:
    print(f"\nError from Claude: {e}")

print("\n\n----- From DeepSeek -----")
try:
    deepseek_response = deepseek_via_openai_client.chat.completions.create(
        model="deepseek-chat",
        messages=prompts,
        stream=True
    )
    for chunk in deepseek_response:
        content = chunk.choices[0].delta.content
        if content:
            print(content, end='', flush=True)
except Exception as e:
    print(f"\nError from DeepSeek: {e}")